# alternative code 01

In [2]:
!pip install -q autogluon.tabular ray==2.10.0

from autogluon.tabular import TabularPredictor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import pickle
import shutil
import glob

warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 21.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.2/266.2 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 76.7 MB/s eta 0:00:00:00:0100:01


In [3]:
class CFG:
    train_path = '/kaggle/input/mental-health-prediction-hackathon/train.csv'
    test_path = '/kaggle/input/mental-health-prediction-hackathon/test.csv'
    sample_sub_path = '/kaggle/input/mental-health-prediction-hackathon/sample_submission.csv'
    
    oof_path = '/kaggle/input/depression-prediction-oof-files'
    
    target = 'Depression'
    n_folds = 5
    seed = 42
    
    time_limit = 3600 * 9

In [4]:
train = pd.read_csv(CFG.train_path, index_col='id')

X = train.drop(CFG.target, axis=1)
y = train[CFG.target]

In [5]:
def get_data(path):
    oof_pred_probs_files = glob.glob(f'{path}/*_oof_pred_probs_*.pkl')
    test_pred_probs_files = glob.glob(f'{path}/*_test_pred_probs_*.pkl')

    oof_pred_probs = pickle.load(open(oof_pred_probs_files[0], 'rb'))
    test_pred_probs = pickle.load(open(test_pred_probs_files[0], 'rb'))
    
    scores = []
    skf = StratifiedKFold(n_splits=CFG.n_folds, random_state=CFG.seed, shuffle=True)
    for _, val_idx in skf.split(X, y):
        y_val = y[val_idx]
        y_pred_probs = oof_pred_probs[val_idx]   
        score = accuracy_score(y_val, y_pred_probs.round())
        scores.append(score)
        
    return oof_pred_probs, test_pred_probs, scores

In [6]:
scores = {}
oof_pred_probs = {}
test_pred_probs = {}

In [7]:
model_paths = glob.glob(f'{CFG.oof_path}/*')
for model_path in model_paths:
    model_name = model_path.split('/')[-1]
    oof_pred_probs[model_name], test_pred_probs[model_name], scores[model_name] = get_data(model_path)

In [8]:
train = pd.DataFrame(oof_pred_probs)
train[CFG.target] = y

test = pd.DataFrame(test_pred_probs)

In [9]:
skf = StratifiedKFold(n_splits=CFG.n_folds, random_state=CFG.seed, shuffle=True)
split = skf.split(train, train[CFG.target])
for i, (_, val_index) in enumerate(split):
    train.loc[val_index, 'fold'] = i

In [10]:
predictor = TabularPredictor(
    problem_type='binary',
    eval_metric='accuracy',
    label=CFG.target,
    groups='fold',
    verbosity=2
)

No path specified. Models will be saved in: "AutogluonModels/ag-20241221_115348"


In [11]:
predictor.fit(
    train_data=train,
    time_limit=CFG.time_limit,
    presets='best_quality',
    ag_args_fit={
        'num_gpus': 1, 
        'num_cpus': 4
    }
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       30.05 GB / 31.35 GB (95.9%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked 

RuntimeError: No models were trained successfully during fit(). Inspect the log output or increase verbosity to determine why no models were fit. Alternatively, set `raise_on_no_models_fitted` to False during the fit call.

In [ ]:
predictor.leaderboard(silent=True).style.background_gradient(subset=['score_val'], cmap='RdYlGn')


In [ ]:
def get_ensemble_weights(predictor):
    info = predictor.info()
    ensemble_weights = {}
    for model_name, values in info["model_info"].items():
        if "Ensemble" in model_name:
            children_info = values["children_info"]
            ensemble_weights[model_name] = values["children_info"][list(children_info.keys())[0]]["model_weights"]
    return ensemble_weights

In [ ]:
ensemble_weights = get_ensemble_weights(predictor)

for key, value in ensemble_weights.items():
    plt.figure(figsize=(8, 8))
    plt.pie(value.values(), labels=value.keys(), autopct='%1.1f%%', colors=sns.color_palette('Set2', len(value)))
    plt.title(key)
    plt.tight_layout()
    plt.show()

In [ ]:
split = StratifiedKFold(n_splits=CFG.n_folds, random_state=CFG.seed, shuffle=True).split(train, train[CFG.target])
for fold_idx, (train_index, val_index) in enumerate(split):
    for model in predictor.model_names():
        model_oof_pred_probs = predictor.predict_proba_oof(model).values[:, 1]
        fold_score = accuracy_score(train.loc[val_index, CFG.target], model_oof_pred_probs[val_index].round())
        if model not in scores:
            scores[model] = []
        scores[model].append(fold_score)

In [ ]:
scores = pd.DataFrame(scores)
mean_scores = scores.mean().sort_values(ascending=False)
order = scores.mean().sort_values(ascending=False).index.tolist()

min_score = mean_scores.min()
max_score = mean_scores.max()
padding = (max_score - min_score) * 0.5
lower_limit = min_score - padding
upper_limit = max_score + padding

fig, axs = plt.subplots(1, 2, figsize=(15, scores.shape[1] * 0.4))

sns.boxplot(data=scores, order=order, ax=axs[0], orient='h', palette='RdYlGn_r')
axs[0].set_title('Fold AUC')
axs[0].set_xlabel('')
axs[0].set_ylabel('')

barplot = sns.barplot(x=mean_scores.values, y=mean_scores.index, ax=axs[1], palette='RdYlGn_r')
axs[1].set_title('Average AUC')
axs[1].set_xlabel('')
axs[1].set_xlim(left=lower_limit, right=upper_limit)
axs[1].set_ylabel('')

for i, score in enumerate(mean_scores.values):
    barplot.text(score, i, round(score, 6), va='center')

plt.tight_layout()
plt.show()

In [ ]:
sub = pd.read_csv(CFG.sample_sub_path)
sub[CFG.target] = predictor.predict(test).values
sub.to_csv(f'sub_autogluon_{np.mean(scores[predictor.model_best]):.6f}.csv', index=False)

In [ ]:
sub
shutil.rmtree("AutogluonModels")

# alternative code 02

In [ ]:
!pip install -q autogluon.tabular ray==2.10.0

In [1]:
# block 1

from autogluon.tabular import TabularPredictor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import pickle
import shutil
import glob

warnings.filterwarnings('ignore')

# block 2

class CFG:
    train_path = '/kaggle/input/mental-health-prediction-hackathon/train.csv'
    test_path = '/kaggle/input/mental-health-prediction-hackathon/test.csv'
    sample_sub_path = '/kaggle/input/mental-health-prediction-hackathon/sample_submission.csv'
    
    oof_path = 'path/to/oof/files'  # Adjust this path to where your OOF files are located
    
    target = 'Depression'
    n_folds = 5
    seed = 42
    
    time_limit = 3600 * 9

# block 3

train = pd.read_csv(CFG.train_path, index_col='id')

X = train.drop(CFG.target, axis=1)
y = train[CFG.target]

# block 4

def get_data(path):
    oof_pred_probs_files = glob.glob(f'{path}/*_oof_pred_probs_*.pkl')
    test_pred_probs_files = glob.glob(f'{path}/*_test_pred_probs_*.pkl')

    oof_pred_probs = pickle.load(open(oof_pred_probs_files[0], 'rb'))
    test_pred_probs = pickle.load(open(test_pred_probs_files[0], 'rb'))
    
    scores = []
    skf = StratifiedKFold(n_splits=CFG.n_folds, random_state=CFG.seed, shuffle=True)
    for _, val_idx in skf.split(X, y):
        y_val = y[val_idx]
        y_pred_probs = oof_pred_probs[val_idx]   
        score = accuracy_score(y_val, y_pred_probs.round())
        scores.append(score)
        
    return oof_pred_probs, test_pred_probs, scores

# block 5

scores = {}
oof_pred_probs = {}
test_pred_probs = {}

# block 6

model_paths = glob.glob(f'{CFG.oof_path}/*')
for model_path in model_paths:
    model_name = model_path.split('/')[-1]
    oof_pred_probs[model_name], test_pred_probs[model_name], scores[model_name] = get_data(model_path)

# block 7

train = pd.DataFrame(oof_pred_probs)
train[CFG.target] = y

test = pd.DataFrame(test_pred_probs)

# block 8

skf = StratifiedKFold(n_splits=CFG.n_folds, random_state=CFG.seed, shuffle=True)
split = skf.split(train, train[CFG.target])
for i, (_, val_index) in enumerate(split):
    train.loc[val_index, 'fold'] = i

# block 9

predictor = TabularPredictor(
    problem_type='binary',
    eval_metric='accuracy',
    label=CFG.target,
    groups='fold',
    verbosity=2
)

# block 10

predictor.fit(
    train_data=train,
    time_limit=CFG.time_limit,
    presets='best_quality',
    ag_args_fit={
        'num_gpus': 1, 
        'num_cpus': 4
    }
)

# block 11

leaderboard = predictor.leaderboard(silent=True).style.background_gradient(subset=['score_val'], cmap='RdYlGn')
print("Leaderboard after training:")
print(leaderboard)

# block 12

def get_ensemble_weights(predictor):
    info = predictor.info()
    ensemble_weights = {}
    for model_name, values in info["model_info"].items():
        if "Ensemble" in model_name:
            children_info = values["children_info"]
            ensemble_weights[model_name] = values["children_info"][list(children_info.keys())[0]]["model_weights"]
    return ensemble_weights

# block 13

ensemble_weights = get_ensemble_weights(predictor)

for key, value in ensemble_weights.items():
    plt.figure(figsize=(8, 8))
    plt.pie(value.values(), labels=value.keys(), autopct='%1.1f%%', colors=sns.color_palette('Set2', len(value)))
    plt.title(key)
    plt.tight_layout()
    plt.show()

# block 14

split = StratifiedKFold(n_splits=CFG.n_folds, random_state=CFG.seed, shuffle=True).split(train, train[CFG.target])
for fold_idx, (train_index, val_index) in enumerate(split):
    for model in predictor.model_names():
        model_oof_pred_probs = predictor.predict_proba_oof(model).values[:, 1]
        fold_score = accuracy_score(train.loc[val_index, CFG.target], model_oof_pred_probs[val_index].round())
        if model not in scores:
            scores[model] = []
        scores[model].append(fold_score)

# block 15

scores = pd.DataFrame(scores)
mean_scores = scores.mean().sort_values(ascending=False)
order = scores.mean().sort_values(ascending=False).index.tolist()

min_score = mean_scores.min()
max_score = mean_scores.max()
padding = (max_score - min_score) * 0.5
lower_limit = min_score - padding
upper_limit = max_score + padding

fig, axs = plt.subplots(1, 2, figsize=(15, scores.shape[1] * 0.4))

sns.boxplot(data=scores, order=order, ax=axs[0], orient='h', palette='RdYlGn_r')
axs[0].set_title('Fold AUC')
axs[0].set_xlabel('')
axs[0].set_ylabel('')

barplot = sns.barplot(x=mean_scores.values, y=mean_scores.index, ax=axs[1], palette='RdYlGn_r')
axs[1].set_title('Average AUC')
axs[1].set_xlabel('')
axs[1].set_xlim(left=lower_limit, right=upper_limit)
axs[1].set_ylabel('')

for i, score in enumerate(mean_scores.values):
    barplot.text(score, i, round(score, 6), va='center')

plt.tight_layout()
plt.show()

# block 16

sub = pd.read_csv(CFG.sample_sub_path)
sub[CFG.target] = predictor.predict(test).values

test_accuracy = accuracy_score(test[CFG.target], sub[CFG.target])
print("Test Accuracy: ", test_accuracy)

sub.to_csv(f'sub_autogluon_{np.mean(scores.mean()):.6f}.csv', index=False)

# block 17

sub
shutil.rmtree("AutogluonModels")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 27.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.2/266.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 89.8 MB/s eta 0:00:00:00:010:01


No path specified. Models will be saved in: "AutogluonModels/ag-20241221_150227"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       29.95 GB / 31.35 GB (95.5%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fi

RuntimeError: No models were trained successfully during fit(). Inspect the log output or increase verbosity to determine why no models were fit. Alternatively, set `raise_on_no_models_fitted` to False during the fit call.

# alternative code 03

In [2]:
!pip install -q autogluon.tabular ray==2.10.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 23.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.2/266.2 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 74.7 MB/s eta 0:00:00:00:0100:01


In [4]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor

# Load datasets
train = pd.read_csv('/kaggle/input/mental-health-prediction-hackathon/train.csv')
test = pd.read_csv('/kaggle/input/mental-health-prediction-hackathon/test.csv')

# Data preprocessing
for feature in train.columns:
    if train[feature].dtype == 'object':
        train[feature] = pd.Categorical(train[feature]).codes
train.fillna(0, inplace=True)

for feature in test.columns:
    if test[feature].dtype == 'object':
        test[feature] = pd.Categorical(test[feature]).codes
test.fillna(0, inplace=True)

# Specify target and features
target = 'Depression'
features = [col for col in train.columns if col not in ['id', 'Name', target]]

# Convert to TabularDataset
train_data = TabularDataset(train)
test_data = TabularDataset(test)

# Set up AutoGluon
predictor = TabularPredictor(label=target, verbosity=3)

# Fit the predictor
predictor.fit(
    train_data=train_data,
    presets='best_quality',
    time_limit=3600,  # 1 hour
    hyperparameters={
        'RF': {},  # Random Forest
        'GBM': {},  # Gradient Boosting Machine
        'CAT': {},  # CatBoost
        'XGB': {},  # XGBoost
        'NN_TORCH': {},  # Neural Network (PyTorch)
        'FASTAI': {},  # FastAI
    }
)

# Evaluate the predictor
performance = predictor.evaluate(train_data)
print("Model Performance on Training Data:", performance)


No path specified. Models will be saved in: "AutogluonModels/ag-20241221_151757"
Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
GPU Count:          0
Memory Avail:       29.38 GB / 31.35 GB (93.7%)
Disk Space Avail:   19.49 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': True, 'num_bag_sets': 1}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': True,
 'calibrate': 'auto',
 'delay_bag_sets': False,
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_callbacks': False,
             'enable_ray_logging': True,
        

Model Performance on Training Data: {'accuracy': 0.9422885572139303, 'balanced_accuracy': 0.8925628513868771, 'mcc': 0.802260583516681, 'roc_auc': 0.9805106050268774, 'f1': 0.8368363943254431, 'precision': 0.8604901028968139, 'recall': 0.8144483122775453}


KeyError: "['Depression'] not found in axis"

In [6]:
# Evaluate the predictor
performance = predictor.evaluate(train_data)
print("Model Performance on Training Data:", performance)

# Make predictions on the test data
predictions = predictor.predict(test_data)

# Prepare the submission file
submission = pd.DataFrame({
    'id': test['id'],
    'Depression': predictions
})
submission.to_csv('team_technoids-autogluon_ensemble_test_predictions.csv', index=False)
print("Result output submission file created successfully!")

Loading: /kaggle/working/AutogluonModels/ag-20241221_151757/models/CatBoost_BAG_L1/model.pkl
Loading: /kaggle/working/AutogluonModels/ag-20241221_151757/models/LightGBM_BAG_L1/model.pkl
Loading: /kaggle/working/AutogluonModels/ag-20241221_151757/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: /kaggle/working/AutogluonModels/ag-20241221_151757/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: /kaggle/working/AutogluonModels/ag-20241221_151757/models/RandomForest_BAG_L1/model.pkl
Loading: /kaggle/working/AutogluonModels/ag-20241221_151757/models/XGBoost_BAG_L1/model.pkl
Loading: /kaggle/working/AutogluonModels/ag-20241221_151757/models/LightGBM_BAG_L2/model.pkl
Loading: /kaggle/working/AutogluonModels/ag-20241221_151757/models/WeightedEnsemble_L3/model.pkl
Loading: /kaggle/working/AutogluonModels/ag-20241221_151757/models/CatBoost_BAG_L1/model.pkl


Model Performance on Training Data: {'accuracy': 0.9422885572139303, 'balanced_accuracy': 0.8925628513868771, 'mcc': 0.802260583516681, 'roc_auc': 0.9805106050268774, 'f1': 0.8368363943254431, 'precision': 0.8604901028968139, 'recall': 0.8144483122775453}


Loading: /kaggle/working/AutogluonModels/ag-20241221_151757/models/LightGBM_BAG_L1/model.pkl
Loading: /kaggle/working/AutogluonModels/ag-20241221_151757/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: /kaggle/working/AutogluonModels/ag-20241221_151757/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: /kaggle/working/AutogluonModels/ag-20241221_151757/models/RandomForest_BAG_L1/model.pkl
Loading: /kaggle/working/AutogluonModels/ag-20241221_151757/models/XGBoost_BAG_L1/model.pkl
Loading: /kaggle/working/AutogluonModels/ag-20241221_151757/models/LightGBM_BAG_L2/model.pkl
Loading: /kaggle/working/AutogluonModels/ag-20241221_151757/models/WeightedEnsemble_L3/model.pkl


Result output submission file created successfully!
